In [2]:

import networkx as nx

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import numpy as np
import pandas as pd
from math import cos, sin, radians
import torch

import os
import shutil
import subprocess
import struct
from glob import glob

import multiprocessing
from functools import partial
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import subprocess

from sklearn.neighbors import NearestNeighbors


In [3]:

def load_points_vtk(file_path):
    points = []
    try:
        with open(file_path, 'rb') as file:
            header = []
            for _ in range(5): # header is 5 lines
                line = file.readline().decode('utf-8').strip()
                header.append(line)
            
            if not header[0].startswith('# vtk DataFile Version'):
                raise ValueError("Non è un file VTK valido")
            
            points_line = header[4]
            if not points_line.startswith('POINTS'):
                raise ValueError("Formato POINTS non trovato")
            
            num_points = int(points_line.split()[1])
            
            for _ in range(num_points):
                point_data = file.read(12)
                x, y, z = struct.unpack('>fff', point_data)
                points.append((x, y, z))
                
    except Exception as e:
        print(f"Errore durante la lettura del file: {str(e)}")
        return []
        
    return points

# file_path = "../.data/referenceCase/constant/polyMesh/points"
# file_path = "../.data/referenceCase/constant/triSurface/rocket.eMesh"
# file_path = "../.data/referenceCase/VTK/rocket/rocket_0.VTK"
file_path = "../.data/referenceCase/VTK/referenceCase_0.VTK"
rocket_file_path = "../.data/referenceCase/VTK/rocket/rocket_0.VTK"
points = load_points_vtk(file_path)
print(len(points))
# print(points)
rocket_points = load_points_vtk(rocket_file_path)
print(len(rocket_points))
# print(points)


50891
848


In [7]:

def plot_3d_graph_with_cut(points, rocket_points=None, connectivity=None, cut_axis=None, cut_value=None, cut_type="above"):

    if cut_axis and cut_value is not None:
        axis_index = {'x': 0, 'y': 1, 'z': 2}[cut_axis]
        if cut_type == "above":
            filtered_points = [p for p in points if p[axis_index] >= cut_value]
            rocket_filtered_points = [p for p in rocket_points if p[axis_index] >= cut_value]
        elif cut_type == "below":
            filtered_points = [p for p in points if p[axis_index] <= cut_value]
            rocket_filtered_points = [p for p in rocket_points if p[axis_index] <= cut_value]
        else:
            raise ValueError("cut_type must be 'above' or 'below'")
    else:
        filtered_points = points

    points_trace = go.Scatter3d(
        x=[point[0] for point in filtered_points],
        y=[point[1] for point in filtered_points],
        z=[point[2] for point in filtered_points],
        mode='markers',
        marker=dict(
            size=1,
            color='blue',
            opacity=0.6
        ),
        name='Mesh Nodes'
    )
    
    rocket_points_trace = go.Scatter3d(
        x=[point[0] for point in rocket_filtered_points],
        y=[point[1] for point in rocket_filtered_points],
        z=[point[2] for point in rocket_filtered_points],
        mode='markers',
        marker=dict(
            size=1,
            color='red',
            opacity=1
        ),
        name='Rocket Nodes'
    )

    edge_x = []
    edge_y = []
    edge_z = []
    
    if connectivity is not None:
        for i in range(connectivity.shape[1]):
            start_node = connectivity[0, i]
            end_node = connectivity[1, i]
            # Only add edges if both nodes are in the filtered points
            if start_node < len(points) and end_node < len(points):
                if points[start_node] in filtered_points and points[end_node] in filtered_points:
                    edge_x.extend([points[start_node][0], points[end_node][0], None])
                    edge_y.extend([points[start_node][1], points[end_node][1], None])
                    edge_z.extend([points[start_node][2], points[end_node][2], None])
        
        edges_trace = go.Scatter3d(
            x=edge_x,
            y=edge_y,
            z=edge_z,
            mode='lines',
            line=dict(
                color='gray',
                width=1
            ),
            opacity=0.3,
            name='Edges'
        )
    
        fig = go.Figure(data=[points_trace, rocket_points_trace, edges_trace])
    else:
        fig = go.Figure(data=[points_trace, rocket_points_trace])
    
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        width=1150,
        height=800,
        showlegend=True
    )
    
    fig.show()

plot_3d_graph_with_cut(points, rocket_points, cut_axis='z', cut_value=5, cut_type="above")
